# - Importing modules and defining functions:

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn import preprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
def typeD(data):
    for c in data.columns:
        if data[c].dtype=='int64' or data[c].dtype=='float64':
            numerical.append(c)
        else:
            categorical.append(c)
            

# - Importing dataset:

In [3]:
dfTrain=pd.read_csv("train.csv")
dtTrain=dfTrain.copy()

dfTest=pd.read_csv("test.csv")
dtTest=dfTest.copy()

# - Exploring dataset:

### *Train data:* 

In [4]:
pd.set_option('display.max_columns', None)
dtTrain.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [5]:
dtTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [6]:
dtTrain.isnull().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

### *Test data:* 

In [7]:
dtTest.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance
0,0x160a,CUS_0xd40,September,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,2022.0,Good,809.98,35.030402,22 Years and 9 Months,No,49.574949,236.64268203272135,Low_spent_Small_value_payments,186.26670208571772
1,0x160b,CUS_0xd40,October,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,9,13.27,4.0,Good,809.98,33.053114,22 Years and 10 Months,No,49.574949,21.465380264657146,High_spent_Medium_value_payments,361.44400385378196
2,0x160c,CUS_0xd40,November,Aaron Maashoh,24,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,4,12.27,4.0,Good,809.98,33.811894,NaN,No,49.574949,148.23393788500925,Low_spent_Medium_value_payments,264.67544623342997
3,0x160d,CUS_0xd40,December,Aaron Maashoh,24_,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",4,5,11.27,4.0,Good,809.98,32.430559,23 Years and 0 Months,No,49.574949,39.08251089460281,High_spent_Medium_value_payments,343.82687322383634
4,0x1616,CUS_0x21b1,September,Rick Rothackerj,28,004-07-5839,_______,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,3,1,5.42,5.0,Good,605.03,25.926822,27 Years and 3 Months,No,18.816215,39.684018417945296,High_spent_Large_value_payments,485.2984336755923


In [8]:
dtTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        50000 non-null  object 
 1   Customer_ID               50000 non-null  object 
 2   Month                     50000 non-null  object 
 3   Name                      44985 non-null  object 
 4   Age                       50000 non-null  object 
 5   SSN                       50000 non-null  object 
 6   Occupation                50000 non-null  object 
 7   Annual_Income             50000 non-null  object 
 8   Monthly_Inhand_Salary     42502 non-null  float64
 9   Num_Bank_Accounts         50000 non-null  int64  
 10  Num_Credit_Card           50000 non-null  int64  
 11  Interest_Rate             50000 non-null  int64  
 12  Num_of_Loan               50000 non-null  object 
 13  Type_of_Loan              44296 non-null  object 
 14  Delay_

In [9]:
dtTest.isnull().sum()

ID                             0
Customer_ID                    0
Month                          0
Name                        5015
Age                            0
SSN                            0
Occupation                     0
Annual_Income                  0
Monthly_Inhand_Salary       7498
Num_Bank_Accounts              0
Num_Credit_Card                0
Interest_Rate                  0
Num_of_Loan                    0
Type_of_Loan                5704
Delay_from_due_date            0
Num_of_Delayed_Payment      3498
Changed_Credit_Limit           0
Num_Credit_Inquiries        1035
Credit_Mix                     0
Outstanding_Debt               0
Credit_Utilization_Ratio       0
Credit_History_Age          4470
Payment_of_Min_Amount          0
Total_EMI_per_month            0
Amount_invested_monthly     2271
Payment_Behaviour              0
Monthly_Balance              562
dtype: int64

# - Cleaning and Preprocessing:

### *Correcting wrong data types:*

In [10]:
lst=['Annual_Income' ,'Num_of_Loan',  'Num_of_Delayed_Payment',  'Changed_Credit_Limit', 'Outstanding_Debt',  'Amount_invested_monthly', 'Monthly_Balance']
for i in lst:
    dtTrain[i]=pd.to_numeric(dtTrain[i],errors = 'coerce')
    dtTest[i]=pd.to_numeric(dtTest[i],errors = 'coerce')

### *Correcting Age and Credit_History_Age values:*

In [11]:
dtTrain['Credit_History_Age'] = dtTrain['Credit_History_Age'].str[:2]+"."+dtTrain['Credit_History_Age'].str[13:15]
dtTest['Credit_History_Age'] = dtTest['Credit_History_Age'].str[:2]+"."+dtTest['Credit_History_Age'].str[13:15]

dtTrain['Credit_History_Age']=pd.to_numeric(dtTrain['Credit_History_Age'],errors = 'coerce')
dtTest['Credit_History_Age']=pd.to_numeric(dtTest['Credit_History_Age'],errors = 'coerce')

dtTrain['Credit_History_Age'].values[dtTrain['Credit_History_Age'] <0 ] = dtTrain['Credit_History_Age'].mean()
dtTest['Credit_History_Age'].values[dtTest['Credit_History_Age'] <0 ] = dtTest['Credit_History_Age'].mean()

dtTrain['Age'] = dtTrain['Age'].str[:2]
dtTest['Age'] = dtTest['Age'].str[:2]

dtTrain['Age']=pd.to_numeric(dtTrain['Age'],errors = 'coerce')
dtTest['Age']=pd.to_numeric(dtTest['Age'],errors = 'coerce')

dtTrain['Age'].values[dtTrain['Age'] <0 ] = dtTrain['Age'].mean()
dtTest['Age'].values[dtTest['Age'] <0 ] = dtTest['Age'].mean()

### *Detecting and removing outliers:*

In [12]:
numerical=[]
categorical=[]
typeD(dtTrain)

In [13]:
def remove_outliers(data):
    z_scores = zscore(data)
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3).all(axis=1)
    data = data[filtered_entries]

In [14]:
remove_outliers(dtTrain.drop(categorical,axis=1))

In [15]:
remove_outliers(dtTest.drop(categorical[:-1],axis=1))

In [16]:
dtTrain.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7.0,11.27,4.0,_,809.98,26.822620,22.1,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.280222,Low_spent_Large_value_payments,284.629162,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,33,821-00-0265,Scientist,19114.12,NaN,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7.0,NaN,4.0,Good,809.98,28.609352,22.3,No,49.574949,81.699521,Low_spent_Medium_value_payments,331.209863,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4.0,6.27,4.0,Good,809.98,31.377862,22.4,No,49.574949,199.458074,Low_spent_Small_value_payments,223.451310,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4.0,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22.5,No,49.574949,41.420153,High_spent_Medium_value_payments,341.489231,Good


### *Replacing NULLs:*

In [17]:
dtTrain.interpolate(method="polynomial", order=2,inplace=True)

In [18]:
dtTest.interpolate(method="polynomial", order=2,inplace=True)

In [19]:
dtTrain.isnull().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   1
Monthly_Inhand_Salary           0
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment          0
Changed_Credit_Limit            0
Num_Credit_Inquiries            0
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age              0
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly         0
Payment_Behaviour               0
Monthly_Balance                 0
Credit_Score                    0
dtype: int64

In [20]:
dtTest.isnull().sum()

ID                             0
Customer_ID                    0
Month                          0
Name                        5015
Age                            0
SSN                            0
Occupation                     0
Annual_Income                  0
Monthly_Inhand_Salary          0
Num_Bank_Accounts              0
Num_Credit_Card                0
Interest_Rate                  0
Num_of_Loan                    0
Type_of_Loan                5704
Delay_from_due_date            0
Num_of_Delayed_Payment         0
Changed_Credit_Limit           0
Num_Credit_Inquiries           0
Credit_Mix                     0
Outstanding_Debt               0
Credit_Utilization_Ratio       0
Credit_History_Age             0
Payment_of_Min_Amount          0
Total_EMI_per_month            0
Amount_invested_monthly        0
Payment_Behaviour              0
Monthly_Balance                0
dtype: int64

In [21]:
dtTrain['Type_of_Loan']=dtTrain['Type_of_Loan'].fillna(method = 'ffill')

In [22]:
dtTest['Type_of_Loan']=dtTest['Type_of_Loan'].fillna(method = 'ffill')

In [23]:
dtTrain=dtTrain.dropna()

In [24]:
dtTest=dtTest.dropna()

In [25]:
dtTrain.isnull().sum()

ID                          0
Customer_ID                 0
Month                       0
Name                        0
Age                         0
SSN                         0
Occupation                  0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Type_of_Loan                0
Delay_from_due_date         0
Num_of_Delayed_Payment      0
Changed_Credit_Limit        0
Num_Credit_Inquiries        0
Credit_Mix                  0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Total_EMI_per_month         0
Amount_invested_monthly     0
Payment_Behaviour           0
Monthly_Balance             0
Credit_Score                0
dtype: int64

In [26]:
dtTest.isnull().sum()

ID                          0
Customer_ID                 0
Month                       0
Name                        0
Age                         0
SSN                         0
Occupation                  0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Type_of_Loan                0
Delay_from_due_date         0
Num_of_Delayed_Payment      0
Changed_Credit_Limit        0
Num_Credit_Inquiries        0
Credit_Mix                  0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Total_EMI_per_month         0
Amount_invested_monthly     0
Payment_Behaviour           0
Monthly_Balance             0
dtype: int64

In [27]:

#imp_num = IterativeImputer(estimator=RandomForestRegressor(),initial_strategy='mean',max_iter=2, random_state=0)

#imp_cat = IterativeImputer(estimator=RandomForestClassifier(),initial_strategy='most_frequent',max_iter=2, random_state=0)
    

In [28]:
#dtTest[numerical] = imp_num.fit_transform(dtTest[numerical])

In [29]:
#dtTest[categorical] = imp_cat.fit_transform(dtTest[categorical[:-1]])

### *Encoding:*

In [30]:
dtTrain['Credit_Mix'].replace(['Bad','Standard','Good'],[1,2,3], inplace=True)
dtTrain['Credit_Mix']=pd.to_numeric(dtTrain['Credit_Mix'],errors = 'coerce')

dtTrain['Payment_of_Min_Amount'].replace(['No','Yes'],[0,1], inplace = True)
dtTrain['Payment_of_Min_Amount']=pd.to_numeric(dtTrain['Payment_of_Min_Amount'],errors = 'coerce')

dtTrain['Month'].replace(['January','February','March','April','May','June','July','August',],[1,2,3,4,5,6,7,8,], inplace=True)
dtTrain['Month']=pd.to_numeric(dtTrain['Month'],errors = 'coerce')

dtTrain = dtTrain[dtTrain['Occupation'] != '_______']
dtTrain['Occupation'].replace(['Scientist','Teacher','Engineer','Entrepreneur','Developer','Lawyer','Media_Manager','Doctor','Journalist', 'Manager', 'Accountant','Musician', 'Mechanic', 'Writer', 'Architect'],[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,], inplace=True)
dtTrain['Occupation']=pd.to_numeric(dtTrain['Occupation'],errors = 'coerce')

dtTrain = dtTrain[dtTrain['Payment_Behaviour'] != '!@9#%8']
dtTrain['Payment_Behaviour'].replace(['High_spent_Small_value_payments', 'Low_spent_Medium_value_payments', 'Low_spent_Small_value_payments', 'High_spent_Medium_value_payments', 'High_spent_Large_value_payments', 'Low_spent_Large_value_payments'],[1,2,3,4,5,6], inplace=True)
dtTrain['Payment_Behaviour']=pd.to_numeric(dtTrain['Payment_Behaviour'],errors = 'coerce')

dtTrain['Credit_Score'].replace(['Poor','Standard','Good'],[1,2,3], inplace=True)
dtTrain['Credit_Score']=pd.to_numeric(dtTrain['Credit_Mix'],errors = 'coerce')


In [31]:
dtTrain['Credit_Score']=dtTrain['Credit_Score'].interpolate(method="linear")


dtTrain.interpolate(method="polynomial", order=2,inplace=True)

In [32]:
dtTrain=dtTrain.dropna()

In [33]:
dtTrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77266 entries, 1 to 99998
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        77266 non-null  object 
 1   Customer_ID               77266 non-null  object 
 2   Month                     77266 non-null  int64  
 3   Name                      77266 non-null  object 
 4   Age                       77266 non-null  int64  
 5   SSN                       77266 non-null  object 
 6   Occupation                77266 non-null  int64  
 7   Annual_Income             77266 non-null  float64
 8   Monthly_Inhand_Salary     77266 non-null  float64
 9   Num_Bank_Accounts         77266 non-null  int64  
 10  Num_Credit_Card           77266 non-null  int64  
 11  Interest_Rate             77266 non-null  int64  
 12  Num_of_Loan               77266 non-null  float64
 13  Type_of_Loan              77266 non-null  object 
 14  Delay_

In [34]:
dtTest['Credit_Mix'].replace(['Bad','Standard','Good'],[1,2,3], inplace=True)
dtTest['Credit_Mix']=pd.to_numeric(dtTest['Credit_Mix'],errors = 'coerce')

dtTest['Payment_of_Min_Amount'].replace(['No','Yes'],[0,1], inplace = True)
dtTest['Payment_of_Min_Amount']=pd.to_numeric(dtTest['Payment_of_Min_Amount'],errors = 'coerce')

dtTest['Month'].replace(['September', 'October' ,'November', 'December'],[9,10,11,12], inplace=True)
dtTest['Month']=pd.to_numeric(dtTest['Month'],errors = 'coerce')

dtTest = dtTest[dtTest['Occupation'] != '_______']
dtTest['Occupation'].replace(['Scientist','Teacher','Engineer','Entrepreneur','Developer','Lawyer','Media_Manager','Doctor','Journalist', 'Manager', 'Accountant','Musician', 'Mechanic', 'Writer', 'Architect'],[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,], inplace=True)
dtTest['Occupation']=pd.to_numeric(dtTest['Occupation'],errors = 'coerce')

dtTest = dtTest[dtTest['Payment_Behaviour'] != '!@9#%8']
dtTest['Payment_Behaviour'].replace(['High_spent_Small_value_payments', 'Low_spent_Medium_value_payments', 'Low_spent_Small_value_payments', 'High_spent_Medium_value_payments', 'High_spent_Large_value_payments', 'Low_spent_Large_value_payments'],[1,2,3,4,5,6], inplace=True)
dtTest['Payment_Behaviour']=pd.to_numeric(dtTest['Payment_Behaviour'],errors = 'coerce')



In [35]:
dtTest.interpolate(method="polynomial", order=2,inplace=True)

In [36]:
dtTest=dtTest.dropna()

In [37]:
dtTest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38658 entries, 0 to 49998
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        38658 non-null  object 
 1   Customer_ID               38658 non-null  object 
 2   Month                     38658 non-null  int64  
 3   Name                      38658 non-null  object 
 4   Age                       38658 non-null  int64  
 5   SSN                       38658 non-null  object 
 6   Occupation                38658 non-null  int64  
 7   Annual_Income             38658 non-null  float64
 8   Monthly_Inhand_Salary     38658 non-null  float64
 9   Num_Bank_Accounts         38658 non-null  int64  
 10  Num_Credit_Card           38658 non-null  int64  
 11  Interest_Rate             38658 non-null  int64  
 12  Num_of_Loan               38658 non-null  float64
 13  Type_of_Loan              38658 non-null  object 
 14  Delay_

In [38]:
numerical=[]
categorical=[]
typeD(dtTrain)

In [39]:
categorical

['ID', 'Customer_ID', 'Name', 'SSN', 'Type_of_Loan']

# - Modeling:

In [40]:
x=dtTrain.drop(['Credit_Score','ID', 'Customer_ID', 'Name', 'SSN', 'Type_of_Loan',],axis=1).values
y=dtTrain['Credit_Score'].values
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.5,random_state=0)
lab_enc = preprocessing.LabelEncoder()
y_train= lab_enc.fit_transform(y_train)

In [41]:
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train))
x_test = pd.DataFrame(scaler.fit_transform(x_test))

In [42]:
model=SVC()
model.fit(x_train, y_train)
model.score(x_train, y_train)


0.9632956280899749

In [43]:
model=DecisionTreeClassifier(max_depth=7,max_features=20)
model.fit(x_train, y_train)
model.score(x_train, y_train)


0.9699220873346621

In [44]:
model=RandomForestClassifier(max_depth=7,max_features=20,n_jobs=7)
model.fit(x_train, y_train)
model.score(x_train, y_train)


0.9740636243625915

In [45]:
categorical

['ID', 'Customer_ID', 'Name', 'SSN', 'Type_of_Loan']

In [46]:
dtTest['Credit_Score']=0
dtTest['Credit_Score']=model.predict(dtTest.drop(['Credit_Score','ID', 'Customer_ID', 'Name', 'SSN', 'Type_of_Loan',],axis=1).values)